In [1]:
import pandas as pd
import os
os.chdir("/media/seconddrive/mta_stationing_problem")
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark import SparkConf
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
# spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
#         .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '80g').master("local[26]")\
#         .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
#         .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
#         .config("spark.sql.autoBroadcastJoinThreshold", -1)\
#         .config("spark.driver.maxResultSize", 0)\
#         .config("spark.shuffle.spill", "true")\
#         .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
#         .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
#         .config("spark.ui.showConsoleProgress", "false")\
#         .getOrCreate()

spark = SparkSession.builder.master("local[*]")\
                .config("spark.sql.session.timeZone", "UTC")\
                .config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC')\
                .config('spark.driver.memory', '2g')\
                .config('spark.executor.memory', '2g')\
                .config("spark.ui.showConsoleProgress", "false")\
                .appName("app").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

22/08/17 23:49:07 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
22/08/17 23:49:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/17 23:49:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/17 23:49:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Merging apc data to service disruption data

In [9]:
# Get APC data
f = os.path.join('data', 'processed', 'apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
get_columns = ['zero_load_at_trip_end', 'stop_sequence', 'stop_id_original', 'stop_name']
get_str = ", ".join([c for c in get_columns])
apcdata.createOrReplaceTempView("apc")

# # filter subset
query = f"""
SELECT {get_str}
FROM apc
ORDER BY arrival_time
"""
apcdata = spark.sql(query)
# apcdata = apcdata.filter(apcdata.zero_load_at_trip_end == 1)
apcdata = apcdata.dropDuplicates(['stop_id_original'])


In [10]:
df = apcdata.toPandas()

In [12]:
df[df['stop_sequence'] == 1].sort_values(by=['stop_id_original'])

,zero_load_at_trip_end,stop_sequence,stop_id_original,stop_name
1272,1.0,1,100OAKS,100 OAKS MALL
667,0.0,1,21WE,21ST AVE PAST WEST END AVE SB
1326,NaN,1,25THYDNM,25TH AVE N & HYDE ST NORTHBOUND MID
33,NaN,1,26ACLANN,26TH AVE N & CLARKSVILLE PIKE NB
1355,1.0,1,7AVHARNN,7TH & HARRSION NB
722,NaN,1,ANDMCAWN,ANDERSON LN & MCARTHUR DR WB
82,NaN,1,ARTS,ARTS MAGNET SCHOOL
99,NaN,1,BILTAPTS,BILTMORE APARTMENTS AT GLASTONBURY & MASSMAN
755,NaN,1,BNA,NASHVILLE INTERNATIONAL AIRPORT
102,NaN,1,BOYBUENN,BOYD DR & BUENAVIEW BLVD NB


In [ ]:
import pandas as pd
import os
os.chdir("/media/seconddrive/mta_stationing_problem")
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark import SparkConf
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

In [ ]:
# spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
#         .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '80g').master("local[26]")\
#         .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
#         .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
#         .config("spark.sql.autoBroadcastJoinThreshold", -1)\
#         .config("spark.driver.maxResultSize", 0)\
#         .config("spark.shuffle.spill", "true")\
#         .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
#         .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
#         .config("spark.ui.showConsoleProgress", "false")\
#         .getOrCreate()

spark = SparkSession.builder.master("local[*]")\
                .config("spark.sql.session.timeZone", "UTC")\
                .config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC')\
                .config('spark.driver.memory', '2g')\
                .config('spark.executor.memory', '2g')\
                .config("spark.ui.showConsoleProgress", "false")\
                .appName("app").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

### Merging apc data to service disruption data

In [ ]:
APC_START = '2020-01-01'
APC_END   = '2022-04-06'

In [ ]:
# Get APC data
f = os.path.join('data', 'processed', 'apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
todelete = apcdata.filter('(load < 0) OR (load IS NULL)').select('transit_date','trip_id','overload_id').distinct()
todelete=todelete.withColumn('marker',F.lit(1))
apcdataafternegdelete=apcdata.join(todelete,on=['trip_id','transit_date','overload_id'],how='left').filter('marker is null').drop('marker')
apcdataafternegdelete = apcdataafternegdelete.sort(['trip_id', 'overload_id'])
get_columns = ['trip_id', 'transit_date', 'arrival_time', 'arrival_time_str', 'vehicle_id',
               'block_abbr', 'stop_sequence', 'stop_id_original', 'overload_id']
get_str = ", ".join([c for c in get_columns])
apcdataafternegdelete.createOrReplaceTempView("apc")

# # filter subset
query = f"""
SELECT {get_str}
FROM apc
ORDER BY arrival_time
"""
apcdata = spark.sql(query)
apcdata = apcdata.dropna(subset=['arrival_time'])
apcdata = apcdata.na.fill(value=-1)
apcdata = apcdata.orderBy("arrival_time")

In [ ]:
# apcdata.groupBy().agg(F.count(F.when(F.col("overload_id")>0, True))).collect()[0][0]
# apcdata.groupBy().agg(F.count(F.when(F.col("overload_id")>0, True))).show()
# +------------------------------------------------+
# |count(CASE WHEN (overload_id > 0) THEN true END)|
# +------------------------------------------------+
# |                                           14371|
# +------------------------------------------------+


In [ ]:
# apcdata.groupBy('overload_id').count().show()
# +-----------+--------+
# |overload_id|   count|
# +-----------+--------+
# |          1|    9295|
# |          4|      47|
# |          2|    4702|
# |          0|15792788|
# |          3|     327|
# +-----------+--------+


In [ ]:
# Get service disruption dataset
fp = os.path.join('data', 'others', 'Service Disruptions_07_2019_08_2022.csv')
disruptions_df = pd.read_csv(fp)
disruptions_df.head()
disruptions_df['DATETIME'] = disruptions_df['DATE'] + ' ' + disruptions_df['TIME']
disruptions_df['DATE'] = pd.to_datetime(disruptions_df['DATE'], format='%m/%d/%y', errors='coerce')
disruptions_df['TIME'] = pd.to_datetime(disruptions_df['TIME'], format='%H:%M:%S', errors='coerce')
disruptions_df['DATETIME'] = pd.to_datetime(disruptions_df['DATETIME'], format='%m/%d/%y %H:%M:%S', errors='coerce')
disruptions_df = disruptions_df[(disruptions_df['DATE'] >= APC_START) & (disruptions_df['DATE'] <= APC_END)]

# Remove weather related disruptions
disruptions_df = disruptions_df[(disruptions_df['REASON'] != 'Weather')].sort_values(by=['DATETIME']).reset_index(drop=True)
print('Shape:', disruptions_df.shape)
disruptions_df = disruptions_df.drop(columns=['COMMENTS'])
disruptions_df['BLOCK'] = disruptions_df['BLOCK'].astype('int32')

# Convert to spark dataframe for merging
disruptions_sp = spark.createDataFrame(disruptions_df)
disruptions_sp = disruptions_sp.withColumn("BLOCK", F.col("BLOCK").cast(IntegerType()))

In [ ]:
disruptions_sp.printSchema()

In [ ]:
apcdata.printSchema()

In [ ]:
def asof_join_backward(l, r):
    return pd.merge_asof(l, r, left_on='DATETIME', right_on='arrival_time', direction="backward", left_by="BLOCK", right_by="block_abbr", tolerance=pd.Timedelta('30d'))

identified_disruptions_backward = disruptions_sp.groupby("BLOCK").cogroup(apcdata.groupby("block_abbr")).applyInPandas(
                                asof_join_backward, schema="""DATE timestamp, TIME timestamp, BLOCK int, REASON string, START_STOP_ABBR string, 
                                                     START_STOP_NAME string, START_STOP_LATITUDE double, START_STOP_LONGITUDE double, DATETIME timestamp,
                                                     trip_id string, transit_date timestamp, arrival_time timestamp, arrival_time_str string, vehicle_id string,
                                                     block_abbr int, stop_sequence int, stop_id_original string, overload_id int""")

def asof_join_forward(l, r):
    return pd.merge_asof(l, r, left_on='DATETIME', right_on='arrival_time', direction="forward", left_by="BLOCK", right_by="block_abbr", tolerance=pd.Timedelta('30d'))

identified_disruptions_forward = disruptions_sp.groupby("BLOCK").cogroup(apcdata.groupby("block_abbr")).applyInPandas(
                                asof_join_forward, schema="""DATE timestamp, TIME timestamp, BLOCK int, REASON string, START_STOP_ABBR string, 
                                                     START_STOP_NAME string, START_STOP_LATITUDE double, START_STOP_LONGITUDE double, DATETIME timestamp,
                                                     trip_id string, transit_date timestamp, arrival_time timestamp, arrival_time_str string, vehicle_id string,
                                                     block_abbr int, stop_sequence int, stop_id_original string, overload_id int""")

In [ ]:
identified_disruptions_backward_df = identified_disruptions_backward.toPandas()
identified_disruptions_forward_df  = identified_disruptions_forward.toPandas()

In [ ]:
cols = ['DATETIME', 'BLOCK', 'REASON', 'START_STOP_ABBR', 
        'trip_id', 'transit_date', 'vehicle_id', 'block_abbr', 'arrival_time', 'stop_sequence', 'stop_id_original', 'overload_id']

In [ ]:
identified_disruptions_backward_df = identified_disruptions_backward_df[cols]
identified_disruptions_forward_df = identified_disruptions_forward_df[cols]

identified_disruptions_df = identified_disruptions_backward_df.merge(identified_disruptions_forward_df, 
                                                                     how='outer', 
                                                                     on=['DATETIME', 'BLOCK', 'REASON', 'START_STOP_ABBR'], 
                                                                     suffixes=('_prev', '_next'))
identified_disruptions_df = identified_disruptions_df.dropna(subset=['trip_id_prev', 'trip_id_next'], how='all')

In [ ]:
identified_disruptions_df

I think many service disruptions here occur at the ends of their trips therefore they have no effect on the succeeding trips.  
No need to send an overload bus since another bus in service can be used to cover? (does this happen?) 

In [ ]:
# fp = os.path.join('data', 'processed', 'matched_service_disruptions.pkl')
# identified_disruptions_df.to_pickle(fp)

In [ ]:
fp = os.path.join('data', 'processed', 'matched_service_disruptions.pkl')
identified_disruptions_df = pd.read_pickle(fp)

In [ ]:
identified_disruptions_df.shape

### Filtering out events where the past stop, next stop and reported disruption all happened on different days

In [ ]:
not_same_day = identified_disruptions_df[(identified_disruptions_df['DATETIME'].dt.date != identified_disruptions_df['transit_date_prev']) & 
                          (identified_disruptions_df['DATETIME'].dt.date != identified_disruptions_df['transit_date_next']) & 
                          (identified_disruptions_df['transit_date_prev'] != identified_disruptions_df['transit_date_next'])].sort_values(by=['DATETIME'])

In [ ]:
# not_same_day[not_same_day['vehicle_id_prev'] == not_same_day['vehicle_id_next']].head()
display(not_same_day.head(1))
not_same_day.shape

In [ ]:
# Checking specific disruptions
def show_slice(disruptions_df, idx):
    disruption_idx = idx
    display(disruptions_df.loc[[disruption_idx]])
    tdf = disruptions_df.loc[disruption_idx]
    disruption_datetime = tdf['DATETIME']
    start_time = tdf['arrival_time_prev'] - pd.Timedelta('7h')
    end_time   = tdf['arrival_time_next'] + pd.Timedelta('7h')
    block      = int(tdf['BLOCK'])
    if pd.isnull(start_time):
        start_time = disruption_datetime
    if pd.isnull(end_time):
        end_time = disruption_datetime
    print(start_time, end_time)
    specific_apcdata = apcdata.filter(F.col("arrival_time").between(start_time, end_time))
    specific_apcdata = specific_apcdata.where(specific_apcdata.	block_abbr == block)
    return specific_apcdata.toPandas()

> Just load all overload data in APC to memory and work on that

In [ ]:
overload_apc = apcdata.where(F.col("overload_id") > 0)
overload_df = overload_apc.toPandas()

In [ ]:
results = {}
pbar = tqdm(total=len(not_same_day))
for k, tdf in not_same_day.iterrows():
    disruption_datetime = tdf['DATETIME']
    start_time = tdf['arrival_time_prev'] - pd.Timedelta('1h')
    end_time   = tdf['arrival_time_next'] + pd.Timedelta('1h')
    block      = int(tdf['BLOCK'])
    tdf = overload_df[(overload_df['block_abbr'] == block)]
    tdf = tdf[(tdf['arrival_time'] >= start_time) & (tdf['arrival_time'] <= end_time)]
    pbar.update(1)
    pbar.set_description(f"Processing {k}:{len(tdf[tdf['overload_id'] > 0])}")
    results[k] = len(tdf)
pbar.close()

In [ ]:
# Disruption reported and vehicle was changed between service days (counted as overload)
data = {'id':list(results.keys()), 'overloads':list(results.values())}
tdf = pd.DataFrame(data)
tdf[tdf['overloads'] > 0].shape

In [ ]:
# Disruption reported but either no APC data or no trip for that same block for the vehicle until a few days after
tdf[tdf['overloads'] == 0].shape

In [ ]:
sdf = show_slice(not_same_day, 174)
with pd.option_context("display.max_rows", 100):
    display(sdf)

In [ ]:
same_day = identified_disruptions_df[(identified_disruptions_df['DATETIME'].dt.date == identified_disruptions_df['transit_date_prev']) & 
                                     (identified_disruptions_df['DATETIME'].dt.date == identified_disruptions_df['transit_date_next'])]
same_day.shape

In [ ]:
results = {}
pbar = tqdm(total=len(same_day))
for k, tdf in same_day.iterrows():
    disruption_datetime = tdf['DATETIME']
    start_time = tdf['arrival_time_prev'] - pd.Timedelta('7h')
    end_time   = tdf['arrival_time_next'] + pd.Timedelta('7h')
    block      = int(tdf['BLOCK'])
    tdf = overload_df[(overload_df['block_abbr'] == block)]
    tdf = tdf[(tdf['arrival_time'] >= start_time) & (tdf['arrival_time'] <= end_time)]
    pbar.update(1)
    pbar.set_description(f"Processing {k}:{len(tdf[tdf['overload_id'] > 0])}")
    results[k] = len(tdf)
pbar.close()

In [ ]:
# Disruption reported and an overload bus was dispatched (our use case)
data = {'id':list(results.keys()), 'overloads':list(results.values())}
tdf = pd.DataFrame(data)
tdf[tdf['overloads'] > 0]
# tdf[tdf['overloads'] > 0].shape

In [ ]:
# Disruption reported but no overload bus needed
tdf[tdf['overloads'] == 0]

In [ ]:
d = {'type':['same_day', 'same_day', 'not_same', 'not_same'], 'overload_status': ['used', 'not used', 'used', 'not_used'], 'count': [75, 604, 13, 2185]}
dd = pd.DataFrame(d)
# dd.plot.bar(stacked=True)
dd.groupby('overload_status').sum().plot(kind='pie', y='count', autopct='%1.0f%%',)

## Overload ID types
* 0: Service bus
* 1: Dispatched for emergency [9295]
* 2: [4702]
* 3: I think used when vehicle is still not able to ply the service
* 4: Used in one trip only `230326` `2020-09-26`

In [ ]:
identified_disruptions_df.loc[603]

In [ ]:
sdf = show_slice(same_day, 603)
with pd.option_context("display.max_rows", 100):
    display(sdf)

Identify how many overload buses were dispatched in the middle of a trip vs just from a depot

In [ ]:
a = same_day.loc[tdf[tdf['overloads'] > 0].id][(same_day['stop_sequence_prev'] == 1) |
                                           (same_day['stop_sequence_next'] == 1)]
same_day.loc[tdf[tdf['overloads'] > 0].id].drop(a.index)

In [ ]:
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
ax = same_day.loc[tdf[tdf['overloads'] > 0].id]['REASON'].value_counts().plot(kind='bar', rot=45)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
plt.xticks(ha='right')

In [ ]:
same_day.loc[tdf[tdf['overloads'] > 0].id][(same_day['vehicle_id_prev'] != same_day['vehicle_id_next'])].shape

Here is the APC data for the trip(s) above. It started the trip but due to disruptions, had to be covered by another vehicle.  
Is vehicle `1913` a special overload vehicle?
* answer is no, so certain in service vehicles can be used to cover for other vehicles.

In [ ]:
trip_id = '245007'
date    = '2021-06-10'
block   = '5605'
specific_apcdata = apcdata.where(F.col("transit_date") == date)
specific_apcdata = specific_apcdata.where(specific_apcdata.trip_id == trip_id)
specific_apcdata = specific_apcdata.where(specific_apcdata.block_abbr == block)
sdf = specific_apcdata.toPandas()

In [ ]:
with pd.option_context("display.max_rows", None):
    display(sdf)

## Checking cases where apc data was initially not null and then null records started showing

In [ ]:
import pandas as pd
import os
os.chdir("/media/seconddrive/mta_stationing_problem")
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark import SparkConf
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

In [ ]:
spark = SparkSession.builder.master("local[*]")\
                .config("spark.sql.session.timeZone", "UTC")\
                .config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC')\
                .config('spark.driver.memory', '2g')\
                .config('spark.executor.memory', '2g')\
                .config("spark.ui.showConsoleProgress", "false")\
                .appName("app").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [ ]:
fp = os.path.join('data', 'processed', 'matched_service_disruptions.pkl')
identified_disruptions_df = pd.read_pickle(fp)
identified_disruptions_df.head()

In [ ]:
not_same_day = identified_disruptions_df[(identified_disruptions_df['DATETIME'].dt.date != identified_disruptions_df['transit_date_prev']) & 
                          (identified_disruptions_df['DATETIME'].dt.date != identified_disruptions_df['transit_date_next']) & 
                          (identified_disruptions_df['transit_date_prev'] != identified_disruptions_df['transit_date_next'])].sort_values(by=['DATETIME'])
# not_same_day.head()

In [ ]:
same_day = identified_disruptions_df[(identified_disruptions_df['DATETIME'].dt.date == identified_disruptions_df['transit_date_prev']) & 
                                     (identified_disruptions_df['DATETIME'].dt.date == identified_disruptions_df['transit_date_next'])]
# same_day[['DATETIME', 'BLOCK', 'trip_id_prev', 'trip_id_next']]
same_day.sort_values(by=['DATETIME']).tail()

In [ ]:
same_day[same_day['trip_id_next'] == same_day['trip_id_prev']]

In [ ]:
# Get APC data
f = os.path.join("data", "apc", "cleaned-wego-daily.apc.parquet")
# f = os.path.join('data', 'processed', 'apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
get_columns = ['trip_id', 'transit_date', 'arrival_time', 'arrival_time_str', 'vehicle_id',
               'block_abbr', 'stop_sequence', 'stop_id_original', 'overload_id']
get_str = ", ".join([c for c in get_columns])
apcdata.createOrReplaceTempView("apc")

# # filter subset
query = f"""
SELECT {get_str}
FROM apc
ORDER BY arrival_time
"""
apcdata = spark.sql(query)
apcdata = apcdata.orderBy("arrival_time")

In [ ]:
# # trip_id = '245007'
# block   = 5605	
# datetime = '2021-06-10 20:15:00'
# date    = datetime.split(' ')[0]

# specific_apcdata = apcdata.where(F.col("transit_date") == date)
# specific_apcdata = specific_apcdata.where(specific_apcdata.block_abbr == block)
# sdf = specific_apcdata.toPandas()

In [ ]:
# trip_ids = sdf.groupby(['trip_id']).agg('last').sort_values('arrival_time')
# trip_ids = trip_ids[trip_ids['arrival_time'] >= datetime].index.tolist()
# new_sdf = []
# for trip_id in trip_ids:
#     new_sdf.append(sdf[sdf['trip_id'] == trip_id].sort_values(by=['stop_sequence']))
# new_sdf = pd.concat(new_sdf).reset_index(drop=True)

# with pd.option_context("display.max_rows", 5):
#     display(new_sdf)

In [ ]:
def get_sdf_status(datetime, block):
    date = datetime.split(' ')[0]

    specific_apcdata = apcdata.where(F.col("transit_date") == date)
    specific_apcdata = specific_apcdata.where(specific_apcdata.block_abbr == block)
    sdf = specific_apcdata.toPandas()
    trip_ids = sdf.groupby(['trip_id']).agg('last').sort_values('arrival_time')
    # Include next trip in case its the overload
    trip_ids = trip_ids[trip_ids['arrival_time'] >= datetime].index.tolist()[0:2]
    new_sdf = []    
    for trip_id in trip_ids:
        # Drop first and last stops (depots)
        new_sdf.append(sdf[sdf['trip_id'] == trip_id].sort_values(by=['stop_sequence']).iloc[2:-2, :])
        
    if len(new_sdf) == 0:
        return -1
    
    new_sdf = pd.concat(new_sdf).reset_index(drop=True)
    new_sdf['arrival_time_str'].fillna(value='NULL', inplace=True)
    new_sdf['is_null'] = pd.isnull(new_sdf['arrival_time'])

    sametrip_overload_multiplier = 1
    if len(new_sdf[(new_sdf['trip_id'] == trip_ids[0])]['overload_id'].unique()) > 1:
        sametrip_overload_multiplier = 10
    
    nexttrip_overload_multiplier = 1
    if len(trip_ids) > 1:
        if len(new_sdf[(new_sdf['trip_id'] == trip_ids[1])]['overload_id'].unique()) > 1:
            nexttrip_overload_multiplier = 10
        
    with pd.option_context("display.max_rows", None):
        display(new_sdf)
        # display(new_sdf[(new_sdf['trip_id'] == trip_ids[0])])
        # display(new_sdf[(new_sdf['trip_id'] == trip_ids[0]) & (new_sdf['overload_id'] == 0)])
        
    # Work on initial trip only and no overload
    new_sdf = new_sdf[(new_sdf['trip_id'] == trip_ids[0]) & (new_sdf['overload_id'] == 0)]

    all_valid = (new_sdf['is_null'] == False).all()
    all_null = (new_sdf['is_null'] == True).all()
    
    first_null = False
    mid_null = False
    latest_null = False
    null_groupby = new_sdf.groupby([(new_sdf.is_null != new_sdf.is_null.shift()).cumsum()])
    ilen = len(null_groupby)
    for i, g in null_groupby:
        if i == 1:
            first_null = g['is_null'].values[0]
        if i > 1 and i < ilen and not mid_null:
            mid_null = g['is_null'].values[0]
        if i == ilen:
            latest_null = g['is_null'].values[0]
    # print(all_null, first_null, mid_null, latest_null, overload_multiplier)
    
    if all_valid:
        return 1 * nexttrip_overload_multiplier * sametrip_overload_multiplier
    if all_null:
        return 2 * nexttrip_overload_multiplier * sametrip_overload_multiplier
    if latest_null:
        return 3 * nexttrip_overload_multiplier * sametrip_overload_multiplier
    if not first_null and mid_null and not latest_null:
        return 4 * nexttrip_overload_multiplier * sametrip_overload_multiplier
    if first_null and not mid_null and not latest_null:
        return 5 * nexttrip_overload_multiplier * sametrip_overload_multiplier


In [ ]:
import swifter
# sd = same_day[same_day['trip_id_next'] == same_day['trip_id_prev']]
sd = same_day
sd['status'] = sd.swifter.apply(lambda x: get_sdf_status(x['DATETIME'].strftime('%Y-%m-%d %H:%M:%S'), x['BLOCK']), axis=1)

In [ ]:
# fp = os.path.join('data', 'processed', 'same_day_same_trip_status.pkl')
# sd.to_pickle(fp)

In [ ]:
fp = os.path.join('data', 'processed', 'same_day_same_trip_status.pkl')
sd = pd.read_pickle(fp)

In [ ]:
statuses = []
# sd = same_day[same_day['trip_id_next'] != same_day['trip_id_prev']]
# sd = sd[sd['status'] == -1]
# for k, row in sd[0:1].iterrows():
for k, row in same_day.loc[[458]].iterrows():
    block = row['BLOCK']
    datetime = row['DATETIME'].strftime('%Y-%m-%d %H:%M:%S')
    status = get_sdf_status(datetime, block)
    print(datetime, k, status)
    statuses.append(status)
    # break
statuses

In [ ]:
fp = os.path.join('data', 'processed', 'same_day_same_trip_status.pkl')
sd = pd.read_pickle(fp)

sd.loc[sd[(sd['status'] == 1) | (sd['status'] == 10) | (sd['status'] == 100)].index, 'merged_status'] = 1
sd.loc[sd[(sd['status'] == 2) | (sd['status'] == 20) | (sd['status'] == 200)].index, 'merged_status'] = 2
sd.loc[sd[(sd['status'] == 3) | (sd['status'] == 30) | (sd['status'] == 300)].index, 'merged_status'] = 3
sd.loc[sd[(sd['status'] == 4) | (sd['status'] == 40) | (sd['status'] == 400)].index, 'merged_status'] = 4
sd.loc[sd[(sd['status'] == 5) | (sd['status'] == 50) | (sd['status'] == 500)].index, 'merged_status'] = 5
sd.loc[sd[sd['status'].isna()].index, 'merged_status'] = 5
sd.loc[sd[sd['status'].isna()].index, 'status'] = 5

sd.loc[sd[(sd['status'] == 1)].index, 'type'] = "no_overload"
sd.loc[sd[(sd['status'] == 2)].index, 'type'] = "no_overload"
sd.loc[sd[(sd['status'] == 3)].index, 'type'] = "no_overload"
sd.loc[sd[(sd['status'] == 4)].index, 'type'] = "no_overload"
sd.loc[sd[(sd['status'] == 5)].index, 'type'] = "no_overload"

sd.loc[sd[(sd['status'] == 10)].index, 'type'] = "nexttrip_overload"
sd.loc[sd[(sd['status'] == 20)].index, 'type'] = "nexttrip_overload"
sd.loc[sd[(sd['status'] == 30)].index, 'type'] = "nexttrip_overload"
sd.loc[sd[(sd['status'] == 40)].index, 'type'] = "nexttrip_overload"
sd.loc[sd[(sd['status'] == 50)].index, 'type'] = "nexttrip_overload"

sd.loc[sd[(sd['status'] == 100)].index, 'type'] = "sametrip_overload"
sd.loc[sd[(sd['status'] == 200)].index, 'type'] = "sametrip_overload"
sd.loc[sd[(sd['status'] == 300)].index, 'type'] = "sametrip_overload"
sd.loc[sd[(sd['status'] == 400)].index, 'type'] = "sametrip_overload"
sd.loc[sd[(sd['status'] == 500)].index, 'type'] = "sametrip_overload"

In [ ]:
sd[sd['type'] != 'no_overload']
df_pivot = pd.pivot_table(sd[sd['type'] != 'no_overload'], values='status', index='REASON', columns='type',
                          aggfunc='count').fillna(0).sort_values('nexttrip_overload', ascending=False)#[['no_overload', 'sametrip_overload', 'nexttrip_overload']]
display(df_pivot)
ax =df_pivot.plot(kind='bar', rot=45, stacked=True)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
plt.xticks(ha='right')
# ax.set_xticklabels(['no null', 'all null', 'null end', 'null mid', 'null start'])
ax.set_xlabel('Trip status')
ax.set_ylabel('Count')
ax.set_title('Handling disruptions')

fp = os.path.join('plots', 'handling_disruptions_REASONS.png')
plt.savefig(fp, dpi=300, bbox_inches='tight')


In [ ]:
df_pivot = pd.pivot_table(sd, values='status', index='merged_status', columns='type',
                          aggfunc='count').fillna(0)[['no_overload', 'sametrip_overload', 'nexttrip_overload']].sort_values('no_overload', ascending=False)

ax = df_pivot.plot(kind='bar', rot=0, stacked=True)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
labels = ['no null', 'all null', 'null end', 'null mid', 'null start']
ax.set_xticklabels([labels[int(i) - 1] for i in df_pivot.index])
ax.set_xlabel('Trip status')
ax.set_ylabel('Count')
ax.set_title('Handling disruptions')

# plt.xticks(ha='right')
fp = os.path.join('plots', 'handling_disruptions.png')
plt.savefig(fp, dpi=300, bbox_inches='tight')

### Handling not same day data

In [ ]:
import swifter
sd = not_same_day
sd['status'] = sd.swifter.apply(lambda x: get_sdf_status(x['DATETIME'].strftime('%Y-%m-%d %H:%M:%S'), x['BLOCK']), axis=1)

In [ ]:
# fp = os.path.join('data', 'processed', 'not_same_day_same_trip_status.pkl')
# sd.to_pickle(fp)

In [ ]:
fp = os.path.join('data', 'processed', 'not_same_day_same_trip_status.pkl')
nsd = pd.read_pickle(fp)
# nsd = nsd[nsd['status'] != -1]

fp = os.path.join('data', 'processed', 'same_day_same_trip_status.pkl')
sd = pd.read_pickle(fp)

In [ ]:
fp = os.path.join('data', 'processed', 'matched_service_disruptions.pkl')
identified_disruptions_df = pd.read_pickle(fp)

In [ ]:
all_disruptions = pd.concat([sd, nsd]).sort_index()

In [ ]:
all_disruptions

In [ ]:
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 1) | (all_disruptions['status'] == 10) | (all_disruptions['status'] == 100)].index, 'merged_status'] = 1
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 2) | (all_disruptions['status'] == 20) | (all_disruptions['status'] == 200)].index, 'merged_status'] = 2
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 3) | (all_disruptions['status'] == 30) | (all_disruptions['status'] == 300)].index, 'merged_status'] = 3
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 4) | (all_disruptions['status'] == 40) | (all_disruptions['status'] == 400)].index, 'merged_status'] = 4
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 5) | (all_disruptions['status'] == 50) | (all_disruptions['status'] == 500)].index, 'merged_status'] = 5
all_disruptions.loc[all_disruptions[all_disruptions['status'].isna()].index, 'merged_status'] = 5
all_disruptions.loc[all_disruptions[all_disruptions['status'].isna()].index, 'status'] = 5

all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 1)].index, 'type'] = "no_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 2)].index, 'type'] = "no_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 3)].index, 'type'] = "no_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 4)].index, 'type'] = "no_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 5)].index, 'type'] = "no_overload"

all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 10)].index, 'type'] = "nexttrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 20)].index, 'type'] = "nexttrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 30)].index, 'type'] = "nexttrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 40)].index, 'type'] = "nexttrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 50)].index, 'type'] = "nexttrip_overload"

all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 100)].index, 'type'] = "sametrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 200)].index, 'type'] = "sametrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 300)].index, 'type'] = "sametrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 400)].index, 'type'] = "sametrip_overload"
all_disruptions.loc[all_disruptions[(all_disruptions['status'] == 500)].index, 'type'] = "sametrip_overload"
all_disruptions.type.unique()

In [ ]:
df_pivot = pd.pivot_table(all_disruptions, values='status', index='merged_status', columns='type',
                          aggfunc='count').fillna(0)[['no_overload', 'sametrip_overload', 'nexttrip_overload']].sort_values('no_overload', ascending=False)

ax = df_pivot.plot(kind='bar', rot=0, stacked=True, color=['#1f77b4', '#ff7f0e',  '#2ca02c'])
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
labels = ['no null', 'all null', 'null end', 'null mid', 'null start']
ax.set_xticklabels([labels[int(i) - 1] for i in df_pivot.index])
ax.set_xlabel('Trip status')
ax.set_ylabel('Count')
ax.set_title('Handling disruptions')

# plt.xticks(ha='right')
fp = os.path.join('plots', 'nsd_handling_disruptions.png')
plt.savefig(fp, dpi=300, bbox_inches='tight')

In [ ]:
df_pivot = pd.pivot_table(all_disruptions[all_disruptions['type'] != 'no_overload'], values='status', index='REASON', columns='type',
                          aggfunc='count').fillna(0).sort_values('nexttrip_overload', ascending=False)#[['no_overload', 'sametrip_overload', 'nexttrip_overload']]
display(df_pivot)
ax = df_pivot.plot(kind='bar', rot=45, stacked=True, color=['#2ca02c', '#ff7f0e'])
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
plt.xticks(ha='right')
# ax.set_xticklabels(['no null', 'all null', 'null end', 'null mid', 'null start'])
ax.set_xlabel('Trip status')
ax.set_ylabel('Count')
ax.set_title('Handling disruptions')

fp = os.path.join('plots', 'handling_disruptions_REASONS.png')
plt.savefig(fp, dpi=300, bbox_inches='tight')


## Adding school schedules

In [ ]:
school_breaks = [#2019-2020
                 ('2019-10-07', '2019-10-11'),
                 ('2019-11-27', '2019-11-29'),
                 ('2019-12-20', '2020-01-06'),
                 ('2020-03-16', '2020-03-20'),
                 ('2020-05-22', '2020-08-04'),
                 # 2020-2021
                 ('2020-10-02', '2020-10-09'),
                 ('2020-11-25', '2020-11-27'),
                 ('2020-12-18', '2021-01-04'),
                 ('2021-03-15', '2021-03-19'),
                 ('2021-05-26', '2021-08-09'),
                 # 2022-2023
                 ('2021-09-02', '2021-09-05'),
                 ('2021-10-07', '2021-10-14'),
                 ('2021-11-23', '2021-11-25'),
                 ('2021-12-19', '2022-01-06'),
                 ('2022-03-13', '2022-03-17'),
                #  ('2022-05-25', '2022-08-09'),
                ]